qwen

sk-1a28c3fcc7e044cbacd6faf47dc89755

firecrawl

fc-343fd362814545f295a89dc14ec4ee09

In [ ]:
#firecrawl
fire_api = "fc-343fd362814545f295a89dc14ec4ee09"

In [ ]:
#Qwen
api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"

In [3]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# Function to fetch and parse the webpage
def fetch_website_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract text from paragraphs or other relevant elements
    paragraphs = soup.find_all('p')
    content = " ".join([p.get_text() for p in paragraphs])
    
    return content

# Fetch the content from the target URL
url = "https://lilianweng.github.io/"
website_content = fetch_website_content(url)

# Initialize the OpenAI client
api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"
client = OpenAI(
    api_key=api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# Pass the website content to the OpenAI model for further processing
completion = client.chat.completions.create(
    model="qwen-plus",  # Example model
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': f"Here is some content from the website: {website_content}. How can I help you with it?"}
    ],
    extra_body={
        "enable_search": True
    }
)

# Print the result
print(completion.model_dump_json())

{"id":"chatcmpl-91733717-3dd7-9158-a828-31565744a0e1","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"It looks like you've provided a comprehensive overview of various topics related to machine learning, particularly focusing on large language models (LLMs), reinforcement learning (RL), and transformer architectures. Here's a structured summary and some key takeaways from the content:\n\n### 1. **Reward Hacking in Reinforcement Learning**\n   - **Definition**: Reward hacking occurs when an RL agent exploits flaws or ambiguities in the reward function to achieve high rewards without genuinely completing the intended task.\n   - **Cause**: Imperfections in RL environments and challenges in specifying accurate reward functions.\n   - **Impact on LLMs**: With the rise of reinforcement learning from human feedback (RLHF) for aligning LLMs, reward hacking has become a critical challenge. Instances include models modifying unit tests to pass coding tasks or 

In [4]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# Function to fetch and parse the webpage
def fetch_website_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract text from paragraphs or other relevant elements
        paragraphs = soup.find_all('p')
        content = " ".join([p.get_text() for p in paragraphs])
        return content
    except Exception as e:
        return f"Error fetching content: {e}"

# Streamlit UI
st.title("Website Content Analyzer")

# Input URL
url = st.text_input("Enter the website URL:", "https://lilianweng.github.io/")

if st.button("Analyze Website"):
    # Fetch content from the provided URL
    website_content = fetch_website_content(url)

    if website_content.startswith("Error"):
        st.error(website_content)
    else:
        # Initialize the OpenAI client
        api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"
        client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        )

        # Pass the website content to the OpenAI model for further processing
        completion = client.chat.completions.create(
            model="qwen-plus",  # Example model
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': f"Here is some content from the website: {website_content}. How can I help you with it?"}
            ],
            extra_body={
                "enable_search": True
            }
        )

        # Display result
        st.subheader("Response from AI")
        st.json(completion.model_dump_json())

2025-02-24 14:14:23.220 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:23.428 
  command:

    streamlit run /Volumes/Outlier/Minerva Agent/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-24 14:14:23.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:23.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:23.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:23.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:23.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:23.430 

In [5]:
import streamlit as st
import requests
import os
import dashscope
from bs4 import BeautifulSoup

# Function to fetch and parse the webpage
def fetch_website_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Find the latest topics by extracting titles and summaries
        # For simplicity, assuming articles are in <article> or <h2> tags. Adjust as needed.
        articles = soup.find_all(['h2', 'article'])

        content = []
        for idx, article in enumerate(articles[:10]):  # Only fetch the latest 10 articles
            title = article.get_text().strip()
            one_liner = article.find_next('p').get_text().strip() if article.find_next('p') else "No summary available"
            content.append(f"{idx + 1}. \"{title}\", \"{one_liner}\", {url}")
        
        return content
    except Exception as e:
        return f"Error fetching content: {e}"

# Function to summarize the content using LLM
def summarize_content(content):
    api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"  # Replace with your actual API key
    # Prepare the messages for summarization
    messages = [
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': f"Please summarize the following content into the latest 10 key ideas in the format: '1. title, one-liner description, website name': {content}"}
    ]
    
    # Call the Dashscope API to summarize the content
    try:
        response = dashscope.Generation.call(
            api_key=api_key,
            model="qwen-plus",  # Example model, replace as needed
            messages=messages,
            enable_search=True,
            result_format='message'
        )

        # Access the response content correctly
        summarized_content = response['message']
        return summarized_content

    except Exception as e:
        print(f"Error: {e}")  # Catch any other errors
        return f"Error summarizing content: {e}"

# Streamlit UI
st.title("Website Content Analyzer and Summarizer")

# URL selection
st.sidebar.header("Select Websites to Analyze")
website_choices = [
    "https://www.qbitai.com/",
    "https://www.jiqizhixin.com/",
    "https://lilianweng.github.io/",
    "https://x.com/deepseek_ai"
]
selected_websites = st.sidebar.multiselect(
    "Choose websites to analyze:",
    website_choices,
    default=website_choices  # Default to analyzing all websites
)

if st.button("Analyze Websites"):
    # Loop through selected websites and fetch content
    all_content = []
    for website in selected_websites:
        st.subheader(f"Latest Topics from {website}")
        website_content = fetch_website_content(website)
        
        if isinstance(website_content, str) and website_content.startswith("Error"):
            st.error(website_content)
        else:
            for topic in website_content:
                st.write(topic)
            all_content.append("\n".join(website_content))
    
    # Combine all content from the websites
    combined_content = "\n\n".join(all_content)
    
    # Summarize the content using LLM
    st.subheader("Summarized Key Ideas from Each Website")
    summarized_content = summarize_content(combined_content)
    st.write(summarized_content)

2025-02-24 14:14:25.173 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.173 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.173 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.174 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.174 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.174 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 14:14:25.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

分区
文章梗概+链接
设置关注词“AI，

进度条

限制时间


总结成1页
各网站详情

general爬虫 = firecrawl
arxiv爬虫
x 爬虫
general爬虫
csdn

https://barretzoph.github.io/
http://joschu.net/blog.html
https://www.csdn.net/?spm=1001.2101.3001.4476


In [6]:
x_api = "68Ht8QWPM68NDGEVqKw4gozGK"
x_api_secret = "UWkfgwI6tryTEtKcJwasYTeYLep5DW8MzVxDEFLQGyUhPJYuRe"
bearer = "AAAAAAAAAAAAAAAAAAAAAIOvzQEAAAAAIu9mJKMErOerEHtcahzfAp5rCtQ%3Do6794kUZOu6ufM1qmsDnTtFYsSIC8WYiSxR5rzOfk4Y31obfM2"
access_token = "1942742353-LmPP6rMN8FhQFBGl5tKPsxaJWm9dhYfjztrV0U3"
access_token_secret = "uTM36nTbShRe5dt00pMz1YJ2NRyURsYKOdorEKVkmeHk2"

In [7]:
import requests
import json
import time
import random
import os

bearer_token = bearer
endpoint_url = "https://api.twitter.com/2/tweets/search/recent"
query_parameters = {
    "query": "from:deepseek_ai",
    "tweet.fields": "id,text,author_id,created_at",
    "max_results": 10,
}
def request_headers(bearer_token: str) -> dict:
    """
    Sets up the request headers. 
    Returns a dictionary summarising the bearer token authentication details.
    """
    return {"Authorization": "Bearer {}".format(bearer_token)}

headers = request_headers(bearer_token)

def connect_to_endpoint(endpoint_url: str, headers: dict, parameters: dict) -> json:
    """
    Connects to the endpoint and requests data.
    Returns a json with Twitter data if a 200 status code is yielded.
    Programme stops if there is a problem with the request and sleeps
    if there is a temporary problem accessing the endpoint.
    """
    response = requests.request(
        "GET", url=endpoint_url, headers=headers, params=parameters
    )
    response_status_code = response.status_code
    if response_status_code != 200:
        if response_status_code >= 400 and response_status_code < 500:
            raise Exception(
                "Cannot get data, the program will stop!\nHTTP {}: {}".format(
                    response_status_code, response.text
                )
            )
        
        sleep_seconds = random.randint(5, 60)
        print(
            "Cannot get data, your program will sleep for {} seconds...\nHTTP {}: {}".format(
                sleep_seconds, response_status_code, response.text
            )
        )
        time.sleep(sleep_seconds)
        return connect_to_endpoint(endpoint_url, headers, parameters)
    return response.json()


json_response = connect_to_endpoint(endpoint_url, headers, query_parameters)

In [9]:
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key= fire_api)

# Crawl a website:
crawl_status = app.crawl_url(
  'https://36kr.com/information/AI/', 
  params={
    'limit': 100, 
    'scrapeOptions': {'formats': ['markdown', 'links']}
  },
  poll_interval=30
)
print(crawl_status)

HTTPError: Payment Required: Failed to start crawl job. Insufficient credits to perform this request. For more credits, you can upgrade your plan at https://firecrawl.dev/pricing or try changing the request limit to a lower value. - No additional error details provided.

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

AttributeError: 'FileFinder' object has no attribute 'find_module'

In [12]:
import requests
import datetime  # Import datetime module
import dashscope

# Function to get raw HTML content from the websites
def get_raw_html(domain):
    try:
        # Send request to the website and get the raw HTML content
        response = requests.get(f'http://{domain}')
        if response.status_code == 200:
            return response.content
        else:
            return f"Failed to retrieve content from {domain}. HTTP Status Code: {response.status_code}"
    except Exception as e:
        return f"Error while fetching content from {domain}: {str(e)}"

# Function to prepare the message for Qwen LLM analysis
def analyze_with_qwen(domain, raw_html):
    messages = [
        {'role': 'system', 'content': 'You are a web researcher. Analyze the raw HTML content and extract key topics in the following format: "1. Title | Description | Website"'},
        {'role': 'user', 'content': f'''
        Analyze the raw HTML content from {domain} and provide the latest 10 topics with:
        1. Article titles in English
        2. Article titles in Chinese
        3. One-line descriptions in English
        4. One-line descriptions in Chinese
        5. Website name
        Use current date: {datetime.date.today()}.
        HTML Content: {raw_html.decode('utf-8')}
        '''}
    ]

    response = dashscope.Generation.call(
        api_key="sk-1a28c3fcc7e044cbacd6faf47dc89755",
        model="qwen-max",
        messages=messages,
        enable_search=True,
        result_format='message'
    )
    return response['output']['choices'][0]['message']['content']

# List of websites to scrape
websites = [
    "lilianweng.github.io",
]

# Scraping and analyzing websites
for site in websites:
    print(f"Scraping raw HTML from {site}...")
    raw_html = get_raw_html(site)
    
    # Check if there was an error
    if isinstance(raw_html, str) and ('Error' in raw_html or 'Failed' in raw_html):
        print(raw_html)
    else:
        print(f"Raw HTML retrieved from {site}. Analyzing with Qwen LLM...\n")
        qwen_analysis = analyze_with_qwen(site, raw_html)
        print(f"Analysis from {site}:\n{qwen_analysis}")
    
    print("\n" + "-" * 50 + "\n")

Scraping raw HTML from lilianweng.github.io...
Raw HTML retrieved from lilianweng.github.io. Analyzing with Qwen LLM...

Analysis from lilianweng.github.io:
Based on the provided HTML content, here are the latest 10 topics from Lilian Weng's blog, formatted as requested. Note that the Chinese translations for titles and descriptions are not available in the provided HTML, so I've used a translation service to provide them.

1. **Reward Hacking in Reinforcement Learning | 强化学习中的奖励黑客**  
   - **Description (EN):** Reward hacking occurs when a reinforcement learning (RL) agent exploits flaws or ambiguities in the reward function to achieve high rewards, without genuinely learning or completing the intended task.  
   - **Description (CN):** 奖励黑客是指强化学习（RL）代理利用奖励函数中的缺陷或模糊性来获得高奖励，而实际上并没有真正学习或完成预期任务。  
   - **Website:** Lil'Log

2. **Extrinsic Hallucinations in LLMs | 大型语言模型中的外在幻觉**  
   - **Description (EN):** Hallucination in large language models usually refers to the model generating unfa

In [ ]:
import streamlit as st
import requests
import datetime
import dashscope

# Function to get raw HTML content from the websites
def get_raw_html(domain):
    try:
        # Send request to the website and get the raw HTML content
        response = requests.get(f'http://{domain}')
        if response.status_code == 200:
            return response.content
        else:
            return f"Failed to retrieve content from {domain}. HTTP Status Code: {response.status_code}"
    except Exception as e:
        return f"Error while fetching content from {domain}: {str(e)}"

# Function to prepare the message for Qwen LLM analysis
def analyze_with_qwen(domain, raw_html):
    messages = [
        {'role': 'system', 'content': 'You are a web researcher. Analyze the raw HTML content and extract key topics in the following format: "1. Title | Description | Website"'},
        {'role': 'user', 'content': f'''
        Analyze the raw HTML content from {domain} and provide the latest 10 topics with:
        1. Article titles in English
        2. Article titles in Chinese
        3. One-line descriptions in English
        4. One-line descriptions in Chinese
        5. Website name
        Use current date: {datetime.date.today()}.
        HTML Content: {raw_html.decode('utf-8')}
        '''}
    ]

    response = dashscope.Generation.call(
        api_key="sk-1a28c3fcc7e044cbacd6faf47dc89755",
        model="qwen-max",
        messages=messages,
        enable_search=True,
        result_format='message'
    )
    return response['output']['choices'][0]['message']['content']

# Streamlit UI components
st.title("Minerva Agent")   

# List of default websites
default_websites = [
    "lilianweng.github.io",
    "qbitai.com",
    "jiqizhixin.com",
    "x.com/deepseek_ai"
]

# Input for user to add websites
input_websites = st.text_area("Website Domains(, Seperated)", 
                              value=', '.join(default_websites), 
                              height=100)

# Convert input string to a list of websites
websites = [site.strip() for site in input_websites.split(',')]

# Display results
for site in websites:
    st.write(f"### Pulling {site}...")
    
    # Get raw HTML
    raw_html = get_raw_html(site)
    
    # Check if there was an error
    if isinstance(raw_html, str) and ('Error' in raw_html or 'Failed' in raw_html):
        st.error(raw_html)
    else:
        st.write(f"Raw HTML retrieved from {site}. Analyzing with Qwen LLM...\n")
        
        # Perform Qwen analysis
        qwen_analysis = analyze_with_qwen(site, raw_html)
        
        # Display results
        st.write(f"### {site} Summary:\n")
        st.text_area(f" {site}", qwen_analysis, height=300)

    st.markdown("\n---\n")